In [1]:
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка предобученной модели и токенизатора BERT
model_name = 'bert-base-uncased'  # можно выбрать другие модели, например, 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
import pandas as pd

In [5]:
dt_avas = pd.read_excel(r'/content/Регламент по пунктам AVAS_v3 (1).xlsx')
dt_wipe = pd.read_excel(r'/content/WINDSCREEN WIPER AND WASHER.xlsx')

In [6]:
dt = pd.concat([dt_avas,dt_wipe],ignore_index = True)

In [7]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        # Используем эмбеддинг токена [CLS], который представляет всё предложение
        return outputs.last_hidden_state[:, 0, :].cpu().numpy()


In [19]:
import spacy
from nltk.corpus import stopwords
import nltk
import re
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def tokenizers(x):
  cleaned_text = re.sub(r'[^\w\s]', '', x)
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(cleaned_text)
  tokens = [token.text for token in doc]
  return tokens


def to_chto(text,stop_words):
  cleaned_text = tokenizers(text)
  dano = [word.lower() for word in cleaned_text if word.lower() not in stop_words]
  text = ' '.join(dano)
  return text

def calculate_cosine_similarity(embedding,target_embedding_2d):
    return cosine_similarity(embedding, target_embedding_2d).flatten()[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
dt['new'] = dt['под-подпункт'].fillna(dt['подпункт']).fillna(' ')

In [54]:
#dk = dt.drop(['emb','preobraz','new'],axis = 1)

In [35]:
dt.drop(0,axis =0,inplace = True)

In [50]:
target = 'AVAS sound starts when moving in R starts (vehicle speed > 0) like it works on other vehicles (especially from China).'
target_embedding = get_embedding(to_chto(target,stop_words))

In [51]:
dk = dt

In [52]:
id = dk.index.to_list()

for i in ['Глава','Под глава', 'подпункт', 'под-подпункт']:

    df = dk.loc[id]  # Используйте loc для работы с индексами

    df['emb'] = (df[i]
                  .apply(lambda x: to_chto(x, stop_words))
                  .apply(lambda x: get_embedding(x.lower())))
    display(df)

    df['similarity'] = df['emb'].apply(lambda x: calculate_cosine_similarity(x, target_embedding))

    display(df)

    # Отображение DataFrame
    # Поиск максимального значения схожести
    max_similarity = df['similarity'].max()

    # Обновление списка индексов для следующей итерации
    id = df[df['similarity'] == max_similarity].index.to_list()


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.883604
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.755312
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.801685
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.809077
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.801960
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.872375
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.820989
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.427672
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.32045263, -0.0923334, -0.20261613, 0.1175...",0.745781
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.883604
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.755312
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.801685
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.809077
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.801960
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.872375
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.820989
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.427672
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...",0.789366
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.883604
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.755312
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.801685
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.809077
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.801960
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...",0.872375
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...",0.820989
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...",0.427672
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
1,6.2.1.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,The test speeds for approval are 10 km/h and 2...,The test speeds for approval are 10 km/h and 2...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.52420247, -0.004788787, 0.24779618, -0.14...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.776326
2,6.2.1.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.4004078, 0.22714819, -0.43724048, 0.26046...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.776326
3,6.2.1.3,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Constant speed tests,If after a vehicle is tested in accordance wit...,If after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...","[[-0.11396836, 0.3120672, 0.2173339, 0.3272216...","[[-0.2579425, 0.14667156, -0.17910647, 0.12582...",0.776326
4,6.2.2.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.17284055, 0.22845581, 0.0063874098, 0.110...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.792055
5,6.2.2.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Reversing test,if after a vehicle is tested in accordance wit...,if after a vehicle is tested in accordance wit...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...","[[-0.12087326, 0.24337976, 0.19177951, 0.32571...","[[-0.59607863, -0.26246428, -0.16164653, -0.05...",0.792055
6,6.2.3.1,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,The intention of frequency shift is to acousti...,The intention of frequency shift is to acousti...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[-0.33915347, 0.14240517, 0.27275795, -0.2697...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...",0.836832
7,6.2.3.2,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Frequency shift to signify acceleration and de...,When tested under the conditions of Annex 3 pa...,When tested under the conditions of Annex 3 pa...,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...","[[0.05346284, 0.1057503, 0.5106856, 0.1541665,...","[[-0.31529218, 0.014327697, 0.15833265, -0.141...",0.836832
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...",0.837539
9,6.2.5,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Driver selectable sounds The vehicle manufa...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0

,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...",0.427672


,index,Номер,Глава,Пункт,Под глава,подпункт,под-подпункт,new,emb_под,emb_Под_глава,emb_подпункт,emb_под-подпункт,emb,similarity
8,6.2.4,6,AVAS,2,Acoustics characteristics\nThe sound emitted b...,Stationary sound\nThe vehicle may emit a sound...,,,"[[-0.32045263, -0.0923334, -0.20261613, 0.1175...","[[-0.5294428, 0.0055162883, 0.5672557, -0.2266...","[[-0.33727434, 0.21388444, 0.17818646, -0.1974...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...","[[-1.0533974, -0.41769162, -0.3749478, -0.1958...",0.394402


In [53]:
#text = ['driving below approximately 32 km/h or while driving in reverse']

In [14]:
#dt.fillna('',axis = 1,inplace=True)

In [25]:
#dt['emb'] = (dt['new']).apply(lambda x: to_chto(x,stop_words)).apply(lambda x: get_embedding(x.lower()))

In [15]:
#dt['emb_под'] = (dt['Глава'].apply(lambda x: to_chto(x,stop_words))).apply(lambda x: get_embedding(x.lower()))
#dt['emb_Под_глава'] = (dt['Под глава'].apply(lambda x: to_chto(x,stop_words))).apply(lambda x: get_embedding(x.lower()))

In [16]:
#dt['emb_подпункт'] = (dt['подпункт'].apply(lambda x: to_chto(x,stop_words))).apply(lambda x: get_embedding(x.lower()))
#dt['emb_под-подпункт'] = (dt['под-подпункт'].apply(lambda x: to_chto(x,stop_words))).apply(lambda x: get_embedding(x.lower()))

In [55]:
#reg_predl = dict()
#for predl in text:
  #emd = to_chto(predl,stop_words)
  #target_embedding = get_embedding(emd)
  #dt['similarity'] = dt['emb'].apply(lambda x: calculate_cosine_similarity(x,target_embedding))
  #top_1 = dt.sort_values(by = 'similarity',ascending = False).reset_index(drop = True)
  #display(top_1)
  #reg_predl[predl] = top_1['new'].to_list()[0]

In [56]:
#dt.drop(0,axis = 0,inplace = True)

In [26]:
#for predl in text:
  #emd = to_chto(predl,stop_words)
  #target_embedding = get_embedding(emd)
  #dt['similarity'] = dt['emb'].apply(lambda x: calculate_cosine_similarity(x,target_embedding))
  #top_1 = dt.sort_values(by = 'similarity',ascending = False).reset_index(drop = True)
  #df = top_1[['index','similarity','Под глава','подпункт','под-подпункт']]
  #df['Offer'] = predl
  #df.to_excel(f'{predl[:20]}.xlsx',index=False)
  #reg_predl[predl] = top_1['new'].to_list()[0]

<ipython-input-26-190b80d73dd8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Offer'] = predl
